In [ ]:
split = "valid"
version = "v1"


In [ ]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
from pathlib import Path


In [ ]:
pipe = pipeline("text2text-generation", model="/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/template_code/bart-dischargeme-results_v2/checkpoint-16500", device=0)

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.", flush=True)


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}", flush=True)
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}", flush=True)
    print_gpu_utilization()

## Data

---

In [ ]:
import pandas as pd
from datasets import Dataset


In [ ]:
print_gpu_utilization()

In [ ]:
# train_data = pd.read_pickle("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/train/discharge_target_with_preceding_text+structured_data.pickle")
# train_ds = Dataset.from_pandas(train_data[['hadm_id', "bhc_preceding_text", "brief_hospital_course"]], split="train")


In [ ]:
if split == "valid":
    fname = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/valid/discharge_target_with_preceding_text+structured_data.pickle"
elif split == "test_phase_1":
    fname = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/test/discharge_target_with_preceding_text+structured_data.pickle"
else:
    raise Exception("Split doesn't exist!")

In [ ]:
data = pd.read_pickle(fname)
ds = Dataset.from_pandas(data[['hadm_id', "bhc_preceding_text", "brief_hospital_course"]], split="test")


In [ ]:
data['brief_hospital_course_word_count'].describe()

In [ ]:
ds

In [ ]:
BATCH_SIZE = 16

In [ ]:
range(14952, ds.num_rows)

In [ ]:
ds.select(range(14576, ds.num_rows))

In [ ]:
%%time
# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
outs = []
for idx, out in enumerate(pipe(KeyDataset(ds.select(range(14576, ds.num_rows)), "bhc_preceding_text"), 
                     batch_size=BATCH_SIZE,
                     clean_up_tokenization_spaces=True,
                     truncation=True,
                     return_text=True,
                     max_length=1028,
                     no_repeat_ngram_size=5, 
                     num_beams=3, 
                     early_stopping=True
                    )):
    print(f"{idx}/920")
    outs.append(out)

In [ ]:
print_gpu_utilization()

In [ ]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [ ]:
outs = flatten(outs)

In [ ]:
data_out = pd.concat([data, pd.DataFrame.from_records(outs).rename({"generated_text": "brief_hospital_course_BART"}, axis=1)], axis=1)

In [ ]:
# data_out.to_pickle(f"/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/bhc_scores/{Path(fname).stem}_BART_gen_{version}.pickle")

In [ ]:
ds.num_rows/16

In [ ]:
16 * 911

# Check results

In [ ]:
out = pd.read_pickle("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/bhc_scores/discharge_target_with_preceding_text+structured_data_BART_gen_v1.pickle")

In [ ]:
out[["brief_hospital_course_BART", 'brief_hospital_course']]

In [ ]:
data_out['brief_hospital_course_BART'].isna().sum()


In [2]:

split = "valid"
version = "v2_test"

import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
from pathlib import Path


pipe = pipeline("text2text-generation", model="/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/template_code/bart-dischargeme-results_v2/checkpoint-16500", device=0)


from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.", flush=True)


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}", flush=True)
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}", flush=True)
    print_gpu_utilization()


import pandas as pd
from datasets import Dataset


print_gpu_utilization()

# train_data = pd.read_pickle("/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/train/discharge_target_with_preceding_text+structured_data.pickle")
# train_ds = Dataset.from_pandas(train_data[['hadm_id', "bhc_preceding_text", "brief_hospital_course"]], split="train")

if split == "valid":
    fname = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/valid/discharge_target_with_preceding_text+structured_data.pickle"
elif split == "test_phase_1":
    fname = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/test_phase_1/discharge_target_with_preceding_text+structured_data.pickle"
else:
    raise Exception("Split doesn't exist!")


data = pd.read_pickle(fname)
ds = Dataset.from_pandas(data[['hadm_id', "bhc_preceding_text", "brief_hospital_course"]], split="test")

print(data['brief_hospital_course_word_count'].describe())



GPU memory occupied: 2086 MB.
count    14719.000000
mean       325.122495
std        238.098437
min         10.000000
25%        160.000000
50%        278.000000
75%        439.000000
max       4414.000000
Name: brief_hospital_course_word_count, dtype: float64


In [6]:
%%time

BATCH_SIZE = 16

# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
outs = []
print(ds)
ds = ds
count = 0
for out in tqdm(pipe(KeyDataset(ds, "bhc_preceding_text"), 
                     batch_size=BATCH_SIZE,
                     clean_up_tokenization_spaces=True,
                     truncation=True,
                     return_text=True,
                     max_length=1024,
                     no_repeat_ngram_size=5, 
                     num_beams=3, 
                     early_stopping=True
                    )):
    outs.append(out)
    count += 1
    if count == 100:
        break
        



Dataset({
    features: ['hadm_id', 'bhc_preceding_text', 'brief_hospital_course'],
    num_rows: 14719
})


  0%|          | 0/920 [00:00<?, ?it/s]

CPU times: user 9min 8s, sys: 1.05 s, total: 9min 9s
Wall time: 9min 15s


In [7]:
print_gpu_utilization()
def flatten(xss):
    return [x for xs in xss for x in xs]

outs = flatten(outs)

data_out = pd.concat([data, pd.DataFrame.from_records(outs).rename({"generated_text": "brief_hospital_course_BART"}, axis=1)], axis=1)


GPU memory occupied: 21360 MB.


In [8]:
data_out.shape[0] - data_out['brief_hospital_course_BART'].isna().sum()

100

In [ ]:

# print("Writing file to:")
# print(f"/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/bhc_scores/{Path(fname).stem}_BART_gen_{version}.pickle")
# data_out.to_pickle(f"/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/bhc_scores/{Path(fname).stem}_BART_gen_{version}.pickle")



In [12]:
import sys
sys.path.insert(0,"../../scoring")

In [15]:
from scoring_bhc import calculate_scores_bhc
from scoring_bhc import compute_overall_score_bhc

In [16]:
from scoring_instructions import calculate_scores_instructions
from scoring_instructions import compute_overall_score_instructions

# Test

In [54]:
bart_results = data_out[~data_out['brief_hospital_course_BART'].isna()]

In [74]:
print(bart_results['bhc_preceding_text'].sample(1).squeeze())

 
Name:  ___                    Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
Heparin Agents / Iodine-Iodine Containing / ACE Inhibitors / 
prednisone / contrast dye
 
Attending: ___.
 
Chief Complaint:
SOB, Chest Pressure
 
Major Surgical or Invasive Procedure:
None

 
History of Present Illness:
Ms ___ is a ___ year old woman with PMHx notable for 
___, PE on coumadin since 1990s, pericardial effusion (___), 
and HIT who presents with chest tightness and sob for the last 3 
days. SHe reports that she stopped her coumadin 7 days ago as 
she had a steroid injection for cervical pain. She reports that 
she does not know what her dry weight is. She weighs herself 
occasionally. In the setting of not feeling well for the last 
couple of days she has not taken her torsemide. She reports that 
the symtpoms are worse with exertion. Reports non productive 
cough, no fevers or chills. She do

In [20]:
reference = bart_results[['hadm_id', "brief_hospital_course"]]
generated = bart_results[['hadm_id', "brief_hospital_course_BART"]].rename({"brief_hospital_course_BART":"brief_hospital_course"}, axis=1)

In [21]:
scores = calculate_scores_bhc(generated,
                 reference,
                 [
                     # "bleu", 
                  "rouge",
                     # "bertscore",
                     # "meteor"
                 ])

Beginning scoring...
rougeScorer initialized
Processed 128/100 samples.
Processed 100/100 samples.
Done.


In [23]:
from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')

In [49]:
bart_results = rouge.compute(predictions=generated["brief_hospital_course"].tolist(), references=reference["brief_hospital_course"].tolist())
print(bart_results)

{'rouge1': 0.3680905356845382, 'rouge2': 0.17954054111488255, 'rougeL': 0.2507297384670023, 'rougeLsum': 0.36203777225555756}


{'rouge1': 0.3680905356845382,
 'rouge2': 0.17954054111488255,
 'rougeL': 0.2507297384670023,
 'rougeLsum': 0.36203777225555756}

In [37]:
import tabulate


print(tabulate.tabulate(bart_results[['brief_hospital_course', "brief_hospital_course_BART"]].sample(10), headers='keys', tablefmt='simple'))



    brief_hospital_course                                                                                                   brief_hospital_course_BART
--  ----------------------------------------------------------------------------------------------------------------------  -------------------------------------------------------------------------------------------------------------------------
25  Mr ___ is a ___ yr old male with hx of                                                                                  ___ with history of treated diffuse large B-cell
    DLBCL s/p 6C of R-CHOP, XRT to meseneteric mass in ___                                                                  lymphoma in ___, s/p 6 cycles of R-CHOP, XRT to
    presenting with new onset abdominal pain, CT scan consistent                                                            mesenteric mass, recently noted to have relapsed
    with widespread lymphadenopathy and mesenteric mass, exploratory                 

In [36]:
with open('sample_BART_results_v1.txt', 'w') as f:
    f.write(tabulate.tabulate(bart_results[['brief_hospital_course', "brief_hospital_course_BART"]].sample(10), headers='keys', tablefmt='simple'))


In [39]:
gpt_output = pd.read_csv("~/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/bhc_submission/comparison_simple_gpt3.5_2shot_2500-3500.csv")

In [40]:
gpt_output

,Unnamed: 0,note_id,hadm_id,input,output,careunit,eventtype,curr_service,gpt_input,gpt_completion
0,2500,16468691-DS-13,27150697,\nName: ___ Unit No: ___\...,## Acute pancreatitis: Ms. ___ is a relatively...,Med/Surg/GYN,transfer,MED,"[{'role': 'system', 'content': 'You are a phys...",Ms. ___ is a ___-year-old woman with a history...
1,2501,15894430-DS-5,29437146,\nName: ___ Unit No: ___\n...,"___ yo ___ female with h/o dementia, hypertens...",Med/Surg/Trauma,transfer,MED,"[{'role': 'system', 'content': 'You are a phys...",Ms. ___ presented with right hip pain after an...
2,2502,11323860-DS-11,29570429,\nName: ___ Unit No: ___\n...,BRIEF SUMMARY\n=============\nMr. ___ is a ___...,Medicine/Cardiology,transfer,CMED,"[{'role': 'system', 'content': 'You are a phys...",Mr. ___ is a ___ y/o male with a history of he...
3,2503,16940938-DS-5,28987319,\nName: ___ Unit No: ___\...,"Mr ___ is an ___ w/ h/o ___ Disease, Alzheimer...",Med/Surg/Trauma,admit,MED,"[{'role': 'system', 'content': 'You are a phys...",Mr. ___ was transferred to the medicine servic...
4,2504,16095232-DS-6,24682082,\nName: ___ Unit No: ___\n...,"___ y/o F w/ IDDM, HTN, alcohol abuse, tobacco...",Medicine,transfer,MED,"[{'role': 'system', 'content': 'You are a phys...",Ms. ___ presented to the ED with dyspnea and a...
...,...,...,...,...,...,...,...,...,...,...
995,3495,13470788-DS-61,22192648,\nName: ___ Unit No: ___\n \n...,"___ with Hep C and EtOH cirrhosis, HCC, s/p li...",Transplant,admit,MED,"[{'role': 'system', 'content': 'You are a phys...",Ms. ___ presented with 1-day history of epigas...
996,3496,13893809-DS-7,24320687,\nName: ___ Unit No: __...,Mr. ___ is a ___ with history of alcohol use d...,Medicine,transfer,MED,"[{'role': 'system', 'content': 'You are a phys...",Mr. ___ presented to the emergency department ...
997,3497,15250779-DS-4,28137597,\nName: ___ Unit No: ___\...,"Mr. ___ (DOB: ___ is a ___ year old male, with...",Medicine/Cardiology,transfer,CMED,"[{'role': 'system', 'content': 'You are a phys...",Mr. ___ presented to the ED after a cardiac ar...
998,3498,12370269-DS-8,26162379,\nName: ___ Unit No: ___\...,The patient was admitted to the General Surgic...,Med/Surg/Trauma,admit,SURG,"[{'role': 'system', 'content': 'You are a phys...",Mrs. ___ presented to the ED with right-sided ...


In [48]:
gpt_results = rouge.compute(predictions=gpt_output["gpt_completion"].tolist()[300:500], references=gpt_output["output"].tolist()[300:500])
print(gpt_results)

{'rouge1': 0.29054295458361973, 'rouge2': 0.06733841617256617, 'rougeL': 0.14695558221582145, 'rougeLsum': 0.278719714358029}


In [52]:
pd.DataFrame.from_records([bart_results, gpt_results], index=['BART', "GPT-3.5"])

,rouge1,rouge2,rougeL,rougeLsum
BART,0.368091,0.179541,0.250730,0.362038
GPT-3.5,0.290543,0.067338,0.146956,0.278720
